<a href="https://colab.research.google.com/github/indumisw-png/Electrical-AI-session/blob/main/House_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
# Imports
import os
import pandas as pd
import numpy as np
from pathlib import Path

# File path (change if your CSV is in another location)
data_path = Path("/content/Housing.csv")

if not data_path.exists():
    raise FileNotFoundError(f"Couldn't find dataset at {data_path}. Upload Housing.csv or change `data_path`.")

# Load
df = pd.read_csv(data_path)
print("Dataset shape:", df.shape)
display(df.head(10))


Dataset shape: (545, 13)


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
5,10850000,7500,3,3,1,yes,no,yes,no,yes,2,yes,semi-furnished
6,10150000,8580,4,3,4,yes,no,no,no,yes,2,yes,semi-furnished
7,10150000,16200,5,3,2,yes,no,no,no,no,0,no,unfurnished
8,9870000,8100,4,1,2,yes,yes,yes,no,yes,2,yes,furnished
9,9800000,5750,3,2,4,yes,yes,no,no,yes,1,yes,unfurnished


In [89]:
# Quick EDA
print("Column types and non-null counts:")
display(df.info())

print("\nDescriptive statistics:")
display(df.describe(include='all').transpose())

print("\nMissing values (descending):")
missing = df.isnull().sum().sort_values(ascending=False)
display(missing[missing>0])


Column types and non-null counts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


None


Descriptive statistics:


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
price,545.0,NaN,NaN,NaN,4766729.247706,1870439.615657,1750000.0,3430000.0,4340000.0,5740000.0,13300000.0
area,545.0,NaN,NaN,NaN,5150.541284,2170.141023,1650.0,3600.0,4600.0,6360.0,16200.0
bedrooms,545.0,NaN,NaN,NaN,2.965138,0.738064,1.0,2.0,3.0,3.0,6.0
bathrooms,545.0,NaN,NaN,NaN,1.286239,0.50247,1.0,1.0,1.0,2.0,4.0
stories,545.0,NaN,NaN,NaN,1.805505,0.867492,1.0,1.0,2.0,2.0,4.0
mainroad,545,2,yes,468,NaN,NaN,NaN,NaN,NaN,NaN,NaN
guestroom,545,2,no,448,NaN,NaN,NaN,NaN,NaN,NaN,NaN
basement,545,2,no,354,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hotwaterheating,545,2,no,520,NaN,NaN,NaN,NaN,NaN,NaN,NaN
airconditioning,545,2,no,373,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Missing values (descending):


,0


In [90]:
# Automatic target detection (common names); fallback to last numeric column
possible_targets = [c for c in df.columns if c.lower() in (
    'price','saleprice','sale_price','house_price','target','median_house_value','amount')]

if len(possible_targets) == 0:
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(numeric_cols) == 0:
        raise ValueError("No numeric columns available to choose as target. Ensure dataset has a numeric price column.")
    target_col = numeric_cols[-1]
    print(f"No common price-like column found — using last numeric column as target: '{target_col}'")
else:
    target_col = possible_targets[0]
    print(f"Detected target column: '{target_col}'")

y = df[target_col].copy()
X = df.drop(columns=[target_col]).copy()


Detected target column: 'price'


In [91]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(include=['object','category','bool']).columns.tolist()

print("Numeric features:", numeric_features)
print("Categorical features:", categorical_features)


Numeric features: ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']
Categorical features: ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']


In [92]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Identify column types
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

# Handle version compatibility
try:
    onehot = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
except TypeError:
    onehot = OneHotEncoder(handle_unknown='ignore', sparse=False)

# Pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', onehot)
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])




In [93]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Pipelines that include preprocessing
models = {
    'LinearRegression': Pipeline([('preprocessor', preprocessor), ('model', LinearRegression())]),
    'RandomForest': Pipeline([('preprocessor', preprocessor), ('model', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))]),
    'GradientBoosting': Pipeline([('preprocessor', preprocessor), ('model', GradientBoostingRegressor(n_estimators=100, random_state=42))])
}

# NOTE: If you want to run hyperparameter search, enable the next optional cell (careful: can be slow).


In [94]:
# Optional: small randomized search for tree models (uncomment to use).
# Warning: tuning will increase runtime. Reduce n_iter and cv if necessary.

# from sklearn.model_selection import RandomizedSearchCV
# import scipy.stats as stats
#
# param_grid_rf = {
#     'model__n_estimators': [100, 150],
#     'model__max_depth': [None, 10, 20],
#     'model__max_features': ['auto', 'sqrt']
# }
#
# param_grid_gb = {
#     'model__n_estimators': [100, 150],
#     'model__learning_rate': [0.05, 0.1],
#     'model__max_depth': [3, 5]
# }
#
# searches = {
#     'RandomForest': RandomizedSearchCV(models['RandomForest'], param_grid_rf, n_iter=4, cv=3, scoring='neg_root_mean_squared_error', random_state=42, n_jobs=-1),
#     'GradientBoosting': RandomizedSearchCV(models['GradientBoosting'], param_grid_gb, n_iter=4, cv=3, scoring='neg_root_mean_squared_error', random_state=42, n_jobs=-1)
# }
#
# for name, rs in searches.items():
#     print("Searching:", name)
#     rs.fit(X_train, y_train)
#     print(name, "best params:", rs.best_params_)
#     # replace the pipeline with best estimator
#     models[name] = rs.best_estimator_


In [95]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

results = {}
fitted_models = {}

for name, pipe in models.items():
    print("Fitting:", name)
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)

    mse = mean_squared_error(y_test, preds)
    rmse = np.sqrt(mse)  # Compute RMSE manually
    mae  = mean_absolute_error(y_test, preds)
    r2   = r2_score(y_test, preds)

    results[name] = {'rmse': rmse, 'mae': mae, 'r2': r2}
    fitted_models[name] = pipe

# Display results
results_df = pd.DataFrame(results).T.reset_index().rename(columns={'index':'model'})
results_df = results_df[['model','rmse','mae','r2']].sort_values('rmse').reset_index(drop=True)
display(results_df)


Fitting: LinearRegression
Fitting: RandomForest
Fitting: GradientBoosting


,model,rmse,mae,r2
0,GradientBoosting,1.302053e+06,9.696879e+05,0.664592
1,LinearRegression,1.324507e+06,9.700434e+05,0.652924
2,RandomForest,1.403925e+06,1.025961e+06,0.610055


In [96]:
import joblib
best = results_df.iloc[0]['model']
print("Best by RMSE:", best)
best_model = fitted_models[best]

# Save model (joblib) into working directory
out_model_path = Path("best_house_price_model.joblib")
joblib.dump(best_model, out_model_path)
print("Saved best model to:", out_model_path.resolve())


Best by RMSE: GradientBoosting
Saved best model to: /content/best_house_price_model.joblib


In [97]:
# Example: load model and predict on new data
import joblib
loaded = joblib.load("best_house_price_model.joblib")
# To predict: prepare a pandas DataFrame with the same feature columns (X.columns) and call:
# preds = loaded.predict(new_df)
print("Loaded model:", type(loaded))


Loaded model: <class 'sklearn.pipeline.Pipeline'>
